In [115]:
import numpy as np
import pandas as pd
import BH_kicks as kicks
import Triple_dynamics as Tr_dyn
import matplotlib.pyplot as plt


from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import z_at_value
from astropy import units as u
from astropy import constants as const

In [48]:
file_path = '/Users/pranavsatheesh/Triples/Github/Triple-Outcomes/Data/'

strong_triples_file_path = file_path+"strong_triples_data_from_ill.csv"

df_triples = pd.read_csv(strong_triples_file_path,index_col=False)
df_triples.columns = df_triples.columns.str.replace("-", "_", regex=True) 

In [49]:
target_value= 9.890662534797627
diff = np.abs(np.log10(df_triples["t_triple_form"]) - target_value)
closest_index = diff.idxmin()
# Print the index and the closest value
print(f"Index of the closest value: {closest_index}")
print(f"Closest value: {np.log10(df_triples['t_triple_form'].loc[closest_index])}")

Index of the closest value: 410
Closest value: 9.890662534797627


In [50]:
df_triples.iloc[410]

M1                      5423034.642281
M2                      2504314.347669
M3                      4808757.494227
qin                           0.461792
qout                          0.606603
t_triple_form        7774322185.205505
z_form                        0.631743
t_form1              5907878613.476105
t_form2              7746696597.111934
t_merge_bin          9553262933.675728
a_2nd_ovtks                   0.027901
bin_merger_flag                    Yes
bhid1              9223372039018758933
bhid2              9223372049574368164
bhid3              9223372037849878320
bhid4              9223372049574368164
f_gas                          0.40031
Vescape                      327.08956
Phi_DM                        0.282227
Phi_star                      0.547906
M_halo               6776279134.845192
M_star               1092773218.260845
rhalf                         3.868777
sigma                         0.050013
rbulge                        1.458332
Name: 410, dtype: object

In [51]:
kicks.a_hard(df_triples.iloc[410]["M1"],df_triples.iloc[410]["qin"])

0.19471043723260845

In [52]:
a_triple = 0.027901

In [53]:
print(np.log10(df_triples.iloc[410]["M1"]),np.log10(df_triples.iloc[410]["M2"]),np.log10(df_triples.iloc[410]["M3"]))

6.7342423787122785 6.3986888416667895 6.682032876143303


In [101]:
a_new = a_triple * (df_triples.iloc[410]["M3"]/df_triples.iloc[410]["M2"])/( 1 + 0.4*(df_triples.iloc[410]["M3"]/(df_triples.iloc[410]["M1"]+df_triples.iloc[410]["M2"])))
print(a_new)

0.043113967268450896


In [55]:
kicks.v_and_a_after_slingshot(df_triples.iloc[410]["M1"],df_triples.iloc[410]["M2"],df_triples.iloc[410]["M3"],df_triples.iloc[410]["a_2nd_ovtks"],False)

(403.67245443724863,
 0.04311450183095533,
 0.8867281534097675,
 10231792.136507109)

In [108]:
a0 = 0.027901
a1 = 0.043113967268450896
m1 = df_triples.iloc[410]["M1"]
m2 = df_triples.iloc[410]["M2"]
m3 = df_triples.iloc[410]["M3"]

In [70]:
np.log10(df_triples.t_triple_form[410])

9.890662534797627

In [56]:
np.log10(df_triples.t_merge_bin[410])

9.980151730961815

In [193]:
np.log10(df_triples.t_merge_bin[410]-df_triples.t_triple_form[410])

9.250161483192995

In [58]:
savepath = '/Users/pranavsatheesh/Triples/Github/Illustris_Data/'
a_dadt_filename = savepath+"a_and_dadt_for_all_strong_triples.npz"
a_dadt_file =np.load(a_dadt_filename)
st_seps = a_dadt_file['st_seps']
st_dadt = a_dadt_file['st_dadt']

In [238]:
np.log10(df_triples.t_triple_form[410]+Tr_dyn.timsecale_to_merger(410,a0*0.9))

10.038674808217012

In [235]:
np.log10(df_triples.t_triple_form[410]+Tr_dyn.timsecale_to_merger(410,a0)*(a1/a0)**4*(m3/m2)*(m1+m3)/(m1+m2))

/Users/pranavsatheesh/Triples/Github/Triple-Outcomes/Triple_dynamics.py:56: RuntimeWarning: divide by zero encountered in divide
  func = 1/dadt
/Users/pranavsatheesh/Triples/Github/Triple-Outcomes/Triple_dynamics.py:47: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  tau = integrate.quad(interp_func,a_triple_after,0)[0]


10.784623443395438

In [98]:
np.log10(z_at_value(cosmo.age,(df_triples.t_merge_bin[410]/10**9)*u.Gyr,zmin=1e-9).value)

-0.41444085089137694

In [161]:
def dadt_gw(m1,m2,a):
    dadt_factor =  ((-64/5)*(const.G**3/const.c**5) * (m1*const.M_sun*m2*const.M_sun*(m1*const.M_sun+m2*const.M_sun))).to(u.pc**4*u.yr**-1)
    return dadt_factor.value * (a)**(-3)

In [ ]:
def dadt_gw_factor(m1, m2):
    # Calculate the constant factor in dadt
    return ((-64/5)*(const.G**3/const.c**5) * 
            (m1*const.M_sun * m2*const.M_sun * (m1*const.M_sun + m2*const.M_sun))
           ).to(u.pc**4 * u.yr**-1).value

def merger_timescale(m1, m2, a0):
    # C is the dadt factor without dependence on a
    C = dadt_gw_factor(m1, m2)
    

In [184]:
def merger_timescale(m1, m2, a0):
    # Calculate the constant factor (in years)
    factor = (5 / 256) * (const.c**5 / (const.G**3 * m1 * const.M_sun * m2 * const.M_sun * (m1 * const.M_sun + m2 * const.M_sun)))
    a0 = (a0*u.pc).to(u.m)
    # Return the timescale in years
    return (factor * a0**4).to(u.yr)

In [185]:
merger_timescale(m1,m2,a0)

<Quantity 3.27202598e+12 yr>

In [230]:
merger_timescale(m1,m2,a1)

<Quantity 1.8655617e+13 yr>

In [231]:
merger_timescale(m1,m3,a1)

<Quantity 7.52734554e+12 yr>

In [201]:
from scipy.interpolate import interp1d

def tinsp_interpolate(sep, dadt):
        '''
        Perform interpolation for finding t inspiral
        interpolates x=sep, y=1/dadt_total
        '''
        func = 1/dadt
        #remove infinities for interpolatio to avoid biased results
        func[func==np.inf] = np.nanmin(func)/10**10  #find a small value compared to minimum
        #perform interpolation
        interp_func = interp1d(sep, func, bounds_error=False, fill_value=0) 
        return interp_func

In [202]:
import scipy as sp

In [205]:
interp_func = tinsp_interpolate(st_seps[410],st_dadt[410])

/var/folders/nd/cy3yj2tx2vlct0d__5r7wcn00000gn/T/ipykernel_36884/1526084891.py:8: RuntimeWarning: divide by zero encountered in divide
  func = 1/dadt


In [206]:
import scipy.integrate as integrate

In [214]:
da = np.diff(st_seps[410])

In [219]:
dts = da/st_dadt[410][:-1]

/var/folders/nd/cy3yj2tx2vlct0d__5r7wcn00000gn/T/ipykernel_36884/2238186084.py:1: RuntimeWarning: invalid value encountered in divide
  dts = da/st_dadt[410][:-1]


In [209]:
np.log10(integrate.quad(interp_func,0.1,0)[0])

/var/folders/nd/cy3yj2tx2vlct0d__5r7wcn00000gn/T/ipykernel_36884/501956783.py:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  np.log10(integrate.quad(interp_func,0.1,0)[0])


9.919526510271417